## 한글 형태소 분석, 시각화, 워드 클라우드 등.
### 형태소 분석, 품사 태깅.
### 데이터 :  샘플 데이터, 페이스북 전자 신문에서 크롤링한 임시 데이터로 대체
### 설치가 조금 복잡함.
### 1) jdk 17이상 2) JPyne(자바, 파이썬 연결하는 도구) 3) konlpy 패키지 설치.

## 주의사항, JPyne 버전 1.4, 파이썬 버전 3.10.7 다운 그레이드
## 시스템 환경 변수, ex) JAVA_HOME : 경로 추가하기. jdk 17